In [2]:
from __future__ import annotations
import zipfile
import pandas as pd
import os
from io import StringIO
import matplotlib.pyplot as plt
import tempfile
import plotly.express as px


In [3]:
import sys
sys.path.append('/Users/alex/Desktop/CS/Internships/DBF/python_tools/')
from util.blackbox import decode_blackbox


In [4]:
BLACKBOX_ZIP = '/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/1_7_2024/Blackbox.zip'
WINDTUNNEL_ZIP = '/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/1_7_2024/WindTunnel.zip'
OUTPUT_DIR = '/Users/alex/Desktop/CS/Internships/DBF/python_tools/data/windTunnel/1_7_2024/processed'

In [5]:
MIN_THROTTLE = 1065
MAX_THROTTLE = 2011

In [11]:
thrust_dfs: dict[str, pd.DataFrame] = {}

with zipfile.ZipFile(WINDTUNNEL_ZIP) as z:
    for name in filter(lambda x: x.endswith(".csv") and not x.startswith('__MACOSX'), z.namelist()):
        data = z.read(name)
        df = pd.read_csv(StringIO(data.decode('utf-8')))
        df = df[df.CODE == 0]
        df.reset_index(inplace=True, drop=True) 
        thrust_dfs[name.replace(".csv","").split("/")[-1].lower()] = df

In [12]:
thrust_dfs.keys()

dict_keys(['at5220_15x14x3_40ms_8s_2.4ah_run1', 'at5220_16x12_30ms_8s_nan_run1', 'at5220_15x14x3_35ms_8s_2.0ah_run1', 'at5220_16x12_35ms_8s_nan_run1', 'at5220_15x14x3_35ms_8s_3.2ah_run1', 'at5220_15x14x3_30ms_8s_2.4ah_run1', 'at5220_15x14x3_static_8s_nan_run1', 'at5220_16x12_40ms_8s_nan_run1', 'at5220_16x12_static_8s_nan_run1', 'at5220_16x12_25ms_8s_nan_run1', 'at5220_15x14x3_25ms_8s_2.4ah_run1'])

In [ ]:
blackbox_dfs: dict[str, pd.DataFrame] = {}

with zipfile.ZipFile(BLACKBOX_ZIP) as z:
    for name in filter(lambda x: x.endswith(".TXT") and not x.startswith('__MACOSX'), z.namelist()):
        with tempfile.TemporaryDirectory() as tmpdirname:
            z.extract(name, tmpdirname)
            df = decode_blackbox(tmpdirname + "/" + name)

        name = name.replace(".TXT", "").split("/")[-1]
        parts = name.split("_")
                
        df.dropna(inplace=True, subset=['motor[0]'])
        df.rename(columns={"motor[0]": "throttle"}, inplace=True)
        df["motor"] = parts[0]
        df["prop"] = parts[1]
        df["airspeed"] = parts[2]
        df["cells"] = parts[3]
        df["battery"] = parts[4]
        df["run"] = parts[5]
        blackbox_dfs[name.lower()] = df

In [14]:
blackbox_dfs.keys()

dict_keys(['at5220_15x14x3_30ms_8s_2.4ah_run1', 'at5220_16x12_40ms_8s_nan_run1', 'at5220_15x14x3_static_8s_nan_run1', 'at5220_16x12_25ms_8s_nan_run1', 'at5220_16x12_static_8s_nan_run1', 'at5220_15x14x3_25ms_8s_2.4ah_run1', 'at5220_15x14x3_35ms_8s_2.0ah_run1', 'at5220_16x12_35ms_8s_nan_run1', 'at5220_15x14x3_40ms_8s_2.4ah_run1', 'at5220_16x12_30ms_8s_nan_run1', 'at5220_15x14x3_35ms_8s_3.2ah_run1'])

In [15]:
# find non common df names
thrust_only_dfs = set(thrust_dfs.keys()).difference(set(blackbox_dfs.keys()))
print(thrust_only_dfs)

blackbox_only_dfs = set(blackbox_dfs.keys()).difference(set(thrust_dfs.keys()))
print(blackbox_only_dfs)

set()
set()


In [17]:
# find common df names
common_dfs = set(thrust_dfs.keys()).intersection(set(blackbox_dfs.keys()))
print(common_dfs)

{'at5220_15x14x3_40ms_8s_2.4ah_run1', 'at5220_15x14x3_30ms_8s_2.4ah_run1', 'at5220_15x14x3_35ms_8s_2.0ah_run1', 'at5220_16x12_25ms_8s_nan_run1', 'at5220_16x12_30ms_8s_nan_run1', 'at5220_16x12_static_8s_nan_run1', 'at5220_16x12_40ms_8s_nan_run1', 'at5220_15x14x3_static_8s_nan_run1', 'at5220_15x14x3_25ms_8s_2.4ah_run1', 'at5220_15x14x3_35ms_8s_3.2ah_run1', 'at5220_16x12_35ms_8s_nan_run1'}


In [20]:
BLACKBOX_COLS = ['throttle', 'escRPM', 'amperage (A)', 'vbat (V)']
THRUST_COLS = ['DYNAMIC PRESSURE', 'Lift', 'Drag']
METADATA = ['motor', 'prop', 'airspeed', 'cells', 'battery', 'run']

for name in common_dfs:
    print(name)
    df = blackbox_dfs[name][BLACKBOX_COLS].copy()
    metadata = blackbox_dfs[name][METADATA].iloc[0]
    bins = df['throttle'].index[(df['throttle'] - df['throttle'].shift(1)).apply(lambda x: x > 20)].tolist()
    # df.plot()
    # for x in bins:
    #     plt.axvline(x = x, color = 'b', label = 'axvline - full height')
    df['throttle_group'] = pd.qcut(df["throttle"], q=[0] + [val / len(df["throttle"]) for val in bins] + [1], labels=list(map(str, bins)) + ["max"])
    df = df.groupby('throttle_group').mean()
    df.reset_index(inplace=True)
    df.drop(columns=['throttle_group'], inplace=True)

    df_thrust = thrust_dfs[name][THRUST_COLS]
    df_thrust['thrust'] = -df_thrust['Drag']

    df = df.join(df_thrust)
    df["percent_throttle"] = (df["throttle"] - MIN_THROTTLE) / (MAX_THROTTLE - MIN_THROTTLE) * 100
    for col in METADATA:
        df[col] = metadata[col]

    df.to_csv(OUTPUT_DIR + "/" + name + ".csv", index=False)

at5220_15x14x3_40ms_8s_2.4ah_run1
at5220_15x14x3_30ms_8s_2.4ah_run1
at5220_15x14x3_35ms_8s_2.0ah_run1
at5220_16x12_25ms_8s_nan_run1
at5220_16x12_30ms_8s_nan_run1
at5220_16x12_static_8s_nan_run1
at5220_16x12_40ms_8s_nan_run1
at5220_15x14x3_static_8s_nan_run1
at5220_15x14x3_25ms_8s_2.4ah_run1
at5220_15x14x3_35ms_8s_3.2ah_run1
at5220_16x12_35ms_8s_nan_run1


/var/folders/gz/pp4ldsnj0tjbngm2gygkp8qc0000gn/T/ipykernel_4415/1193602303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thrust['thrust'] = -df_thrust['Drag']
/var/folders/gz/pp4ldsnj0tjbngm2gygkp8qc0000gn/T/ipykernel_4415/1193602303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thrust['thrust'] = -df_thrust['Drag']
/var/folders/gz/pp4ldsnj0tjbngm2gygkp8qc0000gn/T/ipykernel_4415/1193602303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
